# LoCS Demo

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg', 'pdf') # For export

In [ ]:
def rotmat(angles):
    cos_a = np.cos(angles)
    sin_a = np.sin(angles)
    return np.stack([np.stack([cos_a, -sin_a], -1),
                     np.stack([sin_a, cos_a], -1)], -2)

In [ ]:
colors = ['orangered', 'orange', 'cyan', 'royalblue']
def rot_markers(i, theta=0):
    # The markers below indicate a rectangle, a pentagon, a triangle, and an asterisk with 5 sides
    func_markers = [(4, 0), (5, 0), (3, 0), (5, 1)]
    prior_angles = np.array([0.0, -90.0, -30.0, -90.0])
    return func_markers[i] + tuple([prior_angles[i] + theta])

def get_angle_degrees(x):
    return np.degrees(np.arctan2(x[:, 1], x[:, 0]))

In [ ]:
def plot_with_origin(rows=1, cols=1):
    fig, ax = plt.subplots(rows, cols, squeeze=False, sharex=True, sharey=True,
                          figsize=(4 * cols, 4 * rows))
    for r in range(rows):
        for c in range(cols):
            ax[r, c].axvline(x=0, c="gray", lw=2, alpha=0.5)
            ax[r, c].axhline(y=0, c="gray", lw=2, alpha=0.5)
            ax[r, c].plot((1), (0), ls="", marker=">", ms=10, color="gray", alpha=0.5,
                          transform=ax[r, c].get_yaxis_transform(), clip_on=False)
            ax[r, c].plot((0), (1), ls="", marker="^", ms=10, color="gray", alpha=0.5,
                          transform=ax[r, c].get_xaxis_transform(), clip_on=False)
            ax[r, c].set_axis_off()
    ax[0, 0].set_aspect('equal')
    ax[0, 0].set_xlim([-4.0, 4.0])
    ax[0, 0].set_ylim([-4.0, 4.0])
    return fig, ax

In [ ]:
pos = np.array(
    [[0.7,   0.3],
     [1.2,   2.1],
     [-0.9, -1.4],
     [-1.4,  1.0]]
)

vel = np.array(
    [[0.48, -0.13],
     [-0.03,  0.5],
     [0.43,  0.25],
     [0.45, -0.21]]
)
num_objects = pos.shape[0]
yaw = np.arctan2(vel[:, 1], vel[:, 0])
yaw_angle = np.degrees(yaw)

# In 2D, we could directly do the following:
# R_inv = rotmat(-yaw)
R = rotmat(yaw)
R_inv = R.swapaxes(2, 1)

rot_vel = np.einsum('nmij,nmjk->nmik', R_inv[:, np.newaxis], vel[np.newaxis, :, :, np.newaxis])

rel_pos = pos[None, ...] - pos[:, None, :]
rot_rel_pos = np.einsum('nmij,nmjk->nmik', R_inv[:, np.newaxis], rel_pos[..., np.newaxis])

In [ ]:
fig, ax = plot_with_origin()
for i in range(pos.shape[0]):
    ax[0, 0].plot(pos[i, 0], pos[i, 1], marker=rot_markers(i, yaw_angle[i]),
                  color=colors[i], markeredgecolor='k', markersize=20)
    ax[0, 0].quiver(pos[i, 0], pos[i, 1], vel[i, 0], vel[i, 1], scale=4)
plt.savefig('assets/img/demo_global_frame.png', dpi=300, bbox_inches='tight')

![Global frame](assets/img/demo_global_frame.png)

In [ ]:
fig, ax = plot_with_origin(2, 2)
for i in range(num_objects):
    r, c = divmod(i, 2)
    for j in range(num_objects):
        ax[r, c].plot(rot_rel_pos[i, j, 0], rot_rel_pos[i, j, 1],
                      marker=rot_markers(j, yaw_angle[j]-yaw_angle[i]),
                      color=colors[j], markeredgecolor='k', markersize=20)
        ax[r, c].quiver(rot_rel_pos[i, j, 0], rot_rel_pos[i, j, 1], rot_vel[i, j, 0], rot_vel[i, j, 1], scale=4)
plt.savefig('assets/img/demo_local_frames.png', dpi=300, bbox_inches='tight')

![Local frames](assets/img/demo_local_frames.png)

## Inverse Transformation

In [ ]:
local_pos_pred = np.array(
    [[1.3,  0.2],
     [0.9, -0.1],
     [1.5, -0.2],
     [1.4,  0.0]]
)

local_vel_pred = np.array(
    [[0.05,  0.05],
     [0.0,  -0.05],
     [0.05, -0.05],
     [-0.1,   0.0]]
)

vel_pred_angles = get_angle_degrees(local_vel_pred)

In [ ]:
fig, ax = plot_with_origin(2, 2)
for i in range(num_objects):
    r, c = divmod(i, 2)
    ax[r, c].plot(rot_rel_pos[i, i, 0], rot_rel_pos[i, i, 1],
                  marker=rot_markers(i, 0.0),
                  color=colors[i], markeredgecolor='k', markersize=20, alpha=0.5)
    ax[r, c].quiver(rot_rel_pos[i, i, 0], rot_rel_pos[i, i, 1], rot_vel[i, i, 0], rot_vel[i, i, 1],
                    scale=4, alpha=0.8)
    ax[r, c].plot(local_pos_pred[i, 0], local_pos_pred[i, 1],
                  marker=rot_markers(i, vel_pred_angles[i]),
                  color=colors[i], markeredgecolor='k', markersize=20)
    ax[r, c].quiver(local_pos_pred[i, 0], local_pos_pred[i, 1],
                    rot_vel[i, i, 0]+local_vel_pred[i, 0], rot_vel[i, i, 1]+local_vel_pred[i, 1], scale=4)
plt.savefig('assets/img/demo_local_pred.png', dpi=300, bbox_inches='tight')

![Local Predictions](assets/img/demo_local_pred.png)

In [ ]:
rot_local_pos_pred = np.einsum('nij,njk->nik', R, local_pos_pred[:, :, np.newaxis]).squeeze(-1)
rot_local_vel_pred = np.einsum('nij,njk->nik', R, local_vel_pred[:, :, np.newaxis]).squeeze(-1)

global_pos_pred = pos + rot_local_pos_pred
global_vel_pred = vel + rot_local_vel_pred

global_vel_pred_angles = get_angle_degrees(global_vel_pred)

In [ ]:
fig, ax = plot_with_origin(2, 2)
for i in range(num_objects):
    r, c = divmod(i, 2)
    ax[r, c].plot(pos[i, 0], pos[i, 1], marker=rot_markers(i, 0.0),
                  color=colors[i], markeredgecolor='k', markersize=20, alpha=0.5)
    ax[r, c].quiver(pos[i, 0], pos[i, 1], vel[i, 0], vel[i, 1],
                    scale=4, alpha=0.5)
    ax[r, c].plot(global_pos_pred[i, 0], global_pos_pred[i, 1],
                  marker=rot_markers(i, global_vel_pred_angles[i]),
                  color=colors[i], markeredgecolor='k', markersize=20)
    ax[r, c].quiver(global_pos_pred[i, 0], global_pos_pred[i, 1],
                    global_vel_pred[i, 0], global_vel_pred[i, 1], scale=4)
plt.savefig('assets/img/demo_global_pred.png', dpi=300, bbox_inches='tight')

![Global Predictions Separate](assets/img/demo_global_pred.png)

In [ ]:
fig, ax = plot_with_origin(1, 1)
for i in range(num_objects):
    ax[0, 0].plot(pos[i, 0], pos[i, 1], marker=rot_markers(i, yaw_angle[i]),
                  color=colors[i], markeredgecolor='k', markersize=20, alpha=0.5)
    ax[0, 0].quiver(pos[i, 0], pos[i, 1], vel[i, 0], vel[i, 1],
                    scale=4, alpha=0.5)
    ax[0, 0].plot(global_pos_pred[i, 0], global_pos_pred[i, 1],
                  marker=rot_markers(i, global_vel_pred_angles[i]),
                  color=colors[i], markeredgecolor='k', markersize=20)
    ax[0, 0].quiver(global_pos_pred[i, 0], global_pos_pred[i, 1],
                    global_vel_pred[i, 0], global_vel_pred[i, 1], scale=4)
plt.savefig('assets/img/demo_global_pred_unified.png', dpi=300, bbox_inches='tight')

![Global Predictions Unified](assets/img/demo_global_pred_unified.png)

Hopefully you now have an intuition on how we can construct local coordinate frames and transform object states (positions and velocities$\dagger$) from the global frame to local frames, and how to transform predictions from local frames back to the global frame.

$\dagger$: We can include other states as well, e.g. accelerations, and rotate/translate them accordingly.

In the next demo, we are applying these transformations by building a standalone LoCS network from scratch!